In [32]:
from MetaCriticScraper import MetaCriticScraper
import urllib.request
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd

In [33]:
year = '2020'

In [34]:
hrd = {'User-Agent' : 'Mozilla/5.0', 'referer' : 'https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=' + year + '&distribution=&sort=desc&view=detailed'}
url = "https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=" + year + "&distribution=&sort=desc&view=detailed"

req = requests.get(url, headers=hrd)
# response = urllib.request.urlopen(url)

soup = BeautifulSoup(req.content, "html.parser", from_encoding='utf-8')
# results = soup.select("a")  #id 아래에 strong을 가져오겠다
questions = soup.find("div", {"class":"title_bump"})

d = list(questions.find_all("a", class_="page_num"))
num_list = [str(int(item.string.strip())-1) for item in d]
num_list.insert(0, '0')



In [35]:
num_list

['0', '1', '2', '3', '4', '5', '6', '7', '8', '10']

In [36]:
game_urls = []
for j in range(len(num_list)):
    hrd = {'User-Agent' : 'Mozilla/5.0', 'referer' : "https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=" + year + "&distribution=&sort=desc&view=detailed" + "&page=" + num_list[j] }
    url = "https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=" + year + "&distribution=&sort=desc&view=detailed" + "&page=" + num_list[j]

    req = requests.get(url, headers=hrd)
    # response = urllib.request.urlopen(url)

    soup = BeautifulSoup(req.content, "html.parser", from_encoding='utf-8')
    # results = soup.select("a")  #id 아래에 strong을 가져오겠다
    # 게임 목록을 가져오기 위해 필요한 태그와 클래스를 확인합니다.
    game_items = soup.find_all("td", class_="clamp-summary-wrap")

    # 각 게임별 URL을 추출합니다.
    
    for item in game_items:
        link = item.find("a", class_="title")
        game_urls.append(link['href'])
    """questions = soup.find("div", {"class":"title_bump"})

    a = questions.select("h3")
    name_list = [item.string for item in a]

    b = list(questions.find_all("span", {"class":"data"}))
    platform_list = [item.string.strip() for item in b]

    url_name_list = []
    for i in range(len(name_list)):
        url_name = name_list[i].lower().replace(' ', '-')
        url_name = url_name.replace("'", '')
        url_name = url_name.replace(".", '')
        url_name_list.append(url_name.lower().replace(':', '')) 

    url_platform_list = []
    for i in range(len(platform_list)):
        url_platform_list.append(platform_list[i].lower().replace(' ', '-'))


    url_list = []"""

In [37]:
game_urls

['/game/playstation-4/persona-5-royal',
 '/game/playstation-4/the-last-of-us-part-ii',
 '/game/switch/hades',
 '/game/pc/half-life-alyx',
 '/game/switch/ori-and-the-will-of-the-wisps',
 '/game/pc/hades',
 '/game/playstation-5/demons-souls',
 '/game/xbox-one/dragon-quest-xi-s-echoes-of-an-elusive-age---definitive-edition',
 '/game/pc/microsoft-flight-simulator',
 '/game/pc/crusader-kings-iii',
 '/game/xbox-one/f1-2020',
 '/game/pc/spelunky-2',
 '/game/playstation-4/dragon-quest-xi-s-echoes-of-an-elusive-age---definitive-edition',
 '/game/switch/animal-crossing-new-horizons',
 '/game/pc/factorio',
 '/game/xbox-one/ori-and-the-will-of-the-wisps',
 '/game/xbox-one/yakuza-0',
 '/game/xbox-one/yakuza-like-a-dragon',
 '/game/switch/xenoblade-chronicles-definitive-edition',
 '/game/pc/there-is-no-game-wrong-dimension',
 '/game/playstation-4/dreams',
 '/game/playstation-4/tony-hawks-pro-skater-1-+-2',
 '/game/playstation-5/devil-may-cry-5-special-edition',
 '/game/switch/a-short-hike',
 '/game/

In [38]:
len(game_urls)

992

In [39]:
df = pd.DataFrame(columns=['name', 'platform','publisher', 'rating', 'genre', '#_of_player', 'release_date', 'critic_score', 'critic_count', 'user_score', 'user_count','img_url', 'description'])
for i in game_urls:
    url = 'https://www.metacritic.com' + i
    scraper = MetaCriticScraper(url)
    df = pd.concat([df, pd.DataFrame([{'name' : scraper.game['title'], 'platform': scraper.game['platform'], 'publisher':scraper.game['publisher'],
                        'rating': scraper.game['rating'] , 'genre':scraper.game['genre'] , 
                        '#_of_player':scraper.game['nplayer'] , 'release_date':scraper.game['release_date'] , 'critic_score':scraper.game['critic_score'] , 
                        'critic_count':scraper.game['critic_count'], 'user_score':scraper.game['user_score'] , 'user_count':scraper.game['user_count'], 
                        'img_url':scraper.game['image'] , 'description':scraper.game['description']}])], ignore_index=True)

In [40]:
filename = 'practice_2.csv'  # 파일 이름 생성
df.to_csv('../../data/' + filename, index=False)

In [41]:
df_pr = pd.read_csv('../../data/' + filename)
df_pr.head()

,name,platform,publisher,rating,genre,#_of_player,release_date,critic_score,critic_count,user_score,user_count,img_url,description
0,Persona 5 Royal,PlayStation 4,Atlus,M,"['Role-Playing', 'Japanese-Style']",No Online Multiplayer,"Mar 31, 2020",95,71,8.5,3639.0,https://static.metacritic.com/images/products/...,Prepare for an all-new RPG experience in Perso...
1,The Last of Us Part II,PlayStation 4,SCEA,M,"['General', 'Action Adventure', 'Survival']",No Online Multiplayer,"Jun 19, 2020",93,121,5.8,161654.0,https://static.metacritic.com/images/products/...,Five years after their dangerous journey acros...
2,Hades,Switch,Supergiant Games,T,"['Role-Playing', 'Action Adventure', 'General'...",No Online Multiplayer,"Sep 17, 2020",93,48,9.0,836.0,https://static.metacritic.com/images/products/...,Defy the god of the dead as you hack and slash...
3,Half-Life: Alyx,PC,Valve Software,NaN,"['Action', 'Shooter', 'First-Person', 'Arcade']",NaN,"Mar 23, 2020",93,69,9.2,2807.0,https://static.metacritic.com/images/products/...,Half-Life: Alyx is Valve's VR return to the Ha...
4,Ori and the Will of the Wisps,Switch,Microsoft Game Studios,NaN,NaN,NaN,"Sep 17, 2020",93,24,8.7,469.0,https://static.metacritic.com/images/products/...,From the creators of Ori and the Blind Forest ...


In [42]:
len(df)

992

In [43]:
df_pr.isna().sum()

name              0
platform          0
publisher         3
rating          220
genre             6
#_of_player     232
release_date      3
critic_score      0
critic_count      0
user_score        0
user_count       84
img_url           0
description       0
dtype: int64

In [44]:
df_pr[df_pr['genre'].isna()]

,name,platform,publisher,rating,genre,#_of_player,release_date,critic_score,critic_count,user_score,user_count,img_url,description
4,Ori and the Will of the Wisps,Switch,Microsoft Game Studios,NaN,NaN,NaN,"Sep 17, 2020",93,24,8.7,469.0,https://static.metacritic.com/images/products/...,From the creators of Ori and the Blind Forest ...
15,Ori and the Will of the Wisps,Xbox One,Microsoft Game Studios,NaN,NaN,NaN,"Mar 11, 2020",90,71,8.9,1839.0,https://static.metacritic.com/images/products/...,"The little spirit Ori is no stranger to peril,..."
26,Ori and the Will of the Wisps,PC,Microsoft Game Studios,NaN,NaN,NaN,"Mar 11, 2020",88,44,8.7,1337.0,https://static.metacritic.com/images/products/...,"The little spirit Ori is no stranger to peril,..."
409,Carrion,Switch,Devolver Digital,NaN,NaN,NaN,"Jul 23, 2020",76,28,8.1,44.0,https://static.metacritic.com/images/products/...,In Carrion you’ll assume the role of an amorph...
443,Carrion,Xbox One,Devolver Digital,NaN,NaN,NaN,"Jul 23, 2020",76,17,7.7,60.0,https://static.metacritic.com/images/products/...,In Carrion you’ll assume the role of an amorph...
514,Carrion,PC,Devolver Digital,NaN,NaN,NaN,"Jul 23, 2020",75,57,7.8,282.0,https://static.metacritic.com/images/products/...,In Carrion you’ll assume the role of an amorph...


In [16]:
import numpy as np
df_pr[df_pr['rating'].isna()]

,name,platform,publisher,rating,genre,#_of_player,release_date,critic_score,critic_count,user_score,user_count,img_url,description
3,Tetris Effect: Connected,PlayStation 5,Enhance Games,NaN,"['Puzzle', 'Stacking']",NaN,"Feb 22, 2023",93,8,6.2,47.0,https://static.metacritic.com/images/products/...,Tetris Effect: Connected is Tetris like you've...
7,Resident Evil 4,PC,Capcom,NaN,"['Action Adventure', 'Survival']",NaN,"Mar 24, 2023",92,22,7.4,1509.0,https://static.metacritic.com/images/products/...,SURVIVAL IS JUST THE BEGINNING...Resident Evil...
11,Rogue Legacy 2,PlayStation 5,Cellar Door Games,NaN,"['Role-Playing', 'Roguelike']",NaN,"Jun 20, 2023",90,7,tbd,NaN,https://static.metacritic.com/images/products/...,Rogue Legacy is back! But now it's got a 2 on ...
14,Ghost Trick: Phantom Detective,PC,Capcom,NaN,"['Adventure', 'General']",NaN,"Jun 30, 2023",90,9,5.5,18.0,https://static.metacritic.com/images/products/...,Ghost Trick is a story of mystery and intrigue...
17,Hi-Fi Rush,PC,Bethesda Softworks,NaN,"['Action', 'Platformer', '3D']",NaN,"Jan 25, 2023",89,34,8.8,1317.0,https://static.metacritic.com/images/products/...,Feel the beat as wannabe rockstar Chai and his...
18,Dave the Diver,PC,MINT ROCKET,NaN,"['Action Adventure', 'Adventure', 'General']",NaN,"Jun 28, 2023",89,12,7.5,48.0,https://static.metacritic.com/images/products/...,"Welcome to 'DAVE THE DIVER', a marine adventur..."
19,Pizza Tower,PC,Tour De Pizza,NaN,"['Action', 'General', 'Platformer', '2D']",NaN,"Jan 26, 2023",89,7,8.7,157.0,https://static.metacritic.com/images/products/...,"Peppino Spaghetti, a surprisingly agile and po..."
24,Dead Cells: Return to Castlevania,PC,Motion Twin,NaN,"['Action', 'Platformer', '2D']",NaN,"Mar 6, 2023",88,21,7.5,40.0,https://static.metacritic.com/images/products/...,The inspiration behind it all returns to its 2...
35,Mario Kart 8 Deluxe: Booster Course Pass - Wave 4,Switch,Nintendo,NaN,"['Racing', 'Arcade', 'Automobile']",NaN,"Mar 9, 2023",86,8,7.6,27.0,https://static.metacritic.com/images/products/...,Egg-cellent! Birdo joins the fun in Wave 4 of ...
37,Final Fantasy Pixel Remaster: FF35th Anniversa...,PlayStation 4,Square Enix,NaN,"['Miscellaneous', 'Compilation']",NaN,"May 1, 2023",86,14,8.2,21.0,https://static.metacritic.com/images/products/...,This highly anticipated set includes the physi...


In [17]:
df_pr[df_pr['#_of_player'].isna()]

,name,platform,publisher,rating,genre,#_of_player,release_date,critic_score,critic_count,user_score,user_count,img_url,description
3,Tetris Effect: Connected,PlayStation 5,Enhance Games,NaN,"['Puzzle', 'Stacking']",NaN,"Feb 22, 2023",93,8,6.2,47.0,https://static.metacritic.com/images/products/...,Tetris Effect: Connected is Tetris like you've...
5,Street Fighter 6,PC,Capcom,T,"['Action', 'Fighting', '2D']",NaN,"Jun 2, 2023",92,30,4.6,180.0,https://static.metacritic.com/images/products/...,"Powered by Capcom’s proprietary RE ENGINE, the..."
7,Resident Evil 4,PC,Capcom,NaN,"['Action Adventure', 'Survival']",NaN,"Mar 24, 2023",92,22,7.4,1509.0,https://static.metacritic.com/images/products/...,SURVIVAL IS JUST THE BEGINNING...Resident Evil...
11,Rogue Legacy 2,PlayStation 5,Cellar Door Games,NaN,"['Role-Playing', 'Roguelike']",NaN,"Jun 20, 2023",90,7,tbd,NaN,https://static.metacritic.com/images/products/...,Rogue Legacy is back! But now it's got a 2 on ...
14,Ghost Trick: Phantom Detective,PC,Capcom,NaN,"['Adventure', 'General']",NaN,"Jun 30, 2023",90,9,5.5,18.0,https://static.metacritic.com/images/products/...,Ghost Trick is a story of mystery and intrigue...
17,Hi-Fi Rush,PC,Bethesda Softworks,NaN,"['Action', 'Platformer', '3D']",NaN,"Jan 25, 2023",89,34,8.8,1317.0,https://static.metacritic.com/images/products/...,Feel the beat as wannabe rockstar Chai and his...
18,Dave the Diver,PC,MINT ROCKET,NaN,"['Action Adventure', 'Adventure', 'General']",NaN,"Jun 28, 2023",89,12,7.5,48.0,https://static.metacritic.com/images/products/...,"Welcome to 'DAVE THE DIVER', a marine adventur..."
19,Pizza Tower,PC,Tour De Pizza,NaN,"['Action', 'General', 'Platformer', '2D']",NaN,"Jan 26, 2023",89,7,8.7,157.0,https://static.metacritic.com/images/products/...,"Peppino Spaghetti, a surprisingly agile and po..."
24,Dead Cells: Return to Castlevania,PC,Motion Twin,NaN,"['Action', 'Platformer', '2D']",NaN,"Mar 6, 2023",88,21,7.5,40.0,https://static.metacritic.com/images/products/...,The inspiration behind it all returns to its 2...
34,Diablo IV,PC,Activision,M,"['Role-Playing', 'Action RPG']",NaN,"Jun 6, 2023",87,87,5.1,2321.0,https://static.metacritic.com/images/products/...,MEET YOUR MAKER - Lilith has returned to Sant...
